# Inference with `google/gemma-7b-it`. Fine-tuned using PEFT QLoRA + RAG

## Purpose
This notebook performs inference using the `google/gemma-7b-it` language model from Hugging Face's `transformers` library, fine-tuned using **PEFT (Parameter-Efficient Fine-Tuning)** with **QLoRA** (Quantized Low Rank Adaptation) and enhanced by **Retrieval-Augmented Generation (RAG)**. The test dataset consists of 1363 records, fetched via an HTTP call to the **Palaven API**—a custom API developed exclusively for this thesis. The generated responses, along with evaluation metrics, are saved back to the Palaven API for future analysis.

## Overview

1. **Declaration of Global Variables**:
    - The notebook initializes all necessary global variables, including paths, API credentials, batch sizes, and any configuration settings required for inference and evaluation.

2. **Loading the Fine-tuned Model and Tokenizer**:
    - The `google/gemma-7b-it` model and its corresponding tokenizer are loaded from the Hugging Face library (`transformers`).
    - The model has been fine-tuned using **PEFT** with **QLoRA**, allowing for efficient adaptation to the specific task with reduced memory and computational costs.
    - The **RAG** component is integrated to retrieve relevant external knowledge during inference, enhancing the model's ability to generate more contextually informed responses.

3. **Inference Execution**:
    - **Dataset fetching**: The test dataset, consisting of 1363 records, is retrieved from the cloud via the **Palaven API** using an HTTP request.
    - **Batch processing**: The dataset is processed in batches, with each batch containing 50 instructions. This approach optimizes memory usage and processing time, especially when using a fine-tuned model with retrieval capabilities.
    - **Model Inference**: For each instruction in the dataset, the fine-tuned `google/gemma-7b-it` model, enhanced by the **RAG** component, generates responses by leveraging retrieved external information.
    - **Saving responses**: All generated responses are persisted back to the Palaven API for future evaluation and analysis.

4. **Evaluation Metrics**:
    - **BERTScore**: Precision, Recall, and F1 scores are calculated for each batch to measure the semantic similarity between the generated responses and the ground truth.
    - **ROUGE**: ROUGE 1, ROUGE 2, and ROUGE L scores are computed to evaluate n-gram overlap, with Precision, Recall, and F1 calculated for each metric.
    - **BLEU**: The BLEU score is calculated for each batch, assessing the fluency and correctness of the generated text based on n-gram precision.
    - **Persisting metrics**: All computed metrics are persisted to the cloud via the Palaven API, allowing for further analysis and comparison between experiments (e.g., out-of-the-box vs. fine-tuned models, with and without RAG).


## Key Sections
### 1. Declaration of global variables

| **Variable**             | **Description**                                                                                   | **Value**                                                   |
|--------------------------|---------------------------------------------------------------------------------------------------|-------------------------------------------------------------|
| `hface_read_token`        | Access token for reading from Hugging Face, retrieved from user data.                             | `userdata.get('hface-read-token')`                          |
| `palaven_base_url`        | Base URL for the Palaven API, retrieved from user data.                                           | `userdata.get('palaven-base-url')`                          |
| `batch_size`              | Batch size for inference and evaluation execution.                                                | `50`                                                        |
| `dataset_id`              | Unique identifier for the dataset used in evaluation.                                             | `'F0444B12-5485-4299-B03B-3BDB6D4A2578'`                    |
| `evaluation_session_id`   | Unique identifier for the current evaluation session.                                             | `'EB9C5839-7B20-4D7D-B3F7-17528180676D'`                    |
| `llm_palaven_name`        | Name of the model used in Palaven.                                                 | `'google-gemma'`                                            |
| `llm_model_name`          | Full name of the model used from Hugging Face.                                                    | `'google/gemma-7b-it'`                                      |
| `device_info`             | Information about the device where the inferences and evaluations are executed.                   | `'GPU A100'`                                                |


In [2]:
from google.colab import userdata
from palaven_api_v2 import PalavenApi

hface_read_token = userdata.get('hface-read-token')
palaven_base_url = userdata.get('palaven-base-url')

batch_size = 50
dataset_id = 'F0444B12-5485-4299-B03B-3BDB6D4A2578'
evaluation_session_id = 'EB9C5839-7B20-4D7D-B3F7-17528180676D'
llm_palaven_name = 'google-gemma'
llm_model_name = 'google/gemma-7b-it'
llm_palaven_peft = '/content/drive/MyDrive/Colab Notebooks/palaven/gemma-7b-it-palaven'
device_info = 'GPU A100'

palaven_api = PalavenApi(palaven_base_url)

### 2. Load the base model and the tokenizer

In [ ]:
!pip install -U transformers
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 124.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(llm_model_name, token=hface_read_token)
base_model = AutoModelForCausalLM.from_pretrained(llm_model_name, device_map="auto", token=hface_read_token)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

### 2.1. Load the PEFT QLoRA finetuned adapters and merge them with the base model google/gemma-7b-it

In [ ]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 22.4 MB/s eta 0:00:00


In [ ]:
from peft import PeftModel

peft_model = PeftModel.from_pretrained(base_model, llm_palaven_peft, token=hface_read_token)
model = peft_model.merge_and_unload()
model.to("cuda")

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 3072, padding_idx=0)
    (layers): ModuleList(
      (0-27): 28 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (k_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (v_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=3072, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=3072, out_features=24576, bias=False)
          (up_proj): Linear(in_features=3072, out_features=24576, bias=False)
          (down_proj): Linear(in_features=24576, out_features=3072, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((3072,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((3072,), eps=1

### 3. Inference Execution

In [ ]:
import time
import torch

def get_chat_completion(instruction_prompt, instruction_id, instruction_count):

  start_time = time.time()

  tokenized_instruction = tokenizer(instruction_prompt, return_tensors="pt").to("cuda")

  with torch.no_grad():
    chat_completion_result = model.generate(**tokenized_instruction, max_new_tokens=1000)

  chat_completion = tokenizer.decode(chat_completion_result[0])

  end_time = time.time()
  elapsed_time = end_time - start_time

  print(f'{instruction_count} - LLM-ChatCompletion. InstructionId: {instruction_id},  Elapsed-Time: {elapsed_time:.2f} seconds')

  return chat_completion, elapsed_time

In [ ]:
def add_instruction_to_df(df, instruction_id, instruction):
    df.loc[instruction_id, 'instruction'] = instruction
    df.loc[instruction_id, 'chat_completion'] = None
    df.loc[instruction_id, 'elapsed_time'] = None

In [ ]:
import pandas as pd
import numpy as np
import json

data_shape = {
    'instruction_id': [],
    'instruction': [],
    'chat_completion': [],
    'elapsed_time': []
}

for batch_number in range(1, 29):

  print(f'Start fetching batch {batch_number}')

  instructions = palaven_api.fetch_instruction_test_dataset(evaluation_session_id, batch_number=batch_number, evaluation_exercise='llmfinetunedrag')

  print(f'Start fetching batch done...')

  llm_responses_df = pd.DataFrame(data_shape)
  llm_responses_df.set_index('instruction_id', inplace=True)

  for item in instructions:
    add_instruction_to_df(llm_responses_df, item['instructionId'], item['instruction'])

  instruction_count = 1

  for index, row in llm_responses_df.iterrows():
    instruction = llm_responses_df.loc[index, 'instruction']

    query_augmentation_response = palaven_api.generate_augmented_prompt(llm_palaven_name, instruction)
    instruction_prompt = query_augmentation_response['prompt']

    instruction_id = int(index)

    chat_completion, elapsed_time = get_chat_completion(instruction_prompt, instruction_id, instruction_count)
    palaven_api.save_model_response(evaluation_session_id, 'llmfinetunedrag', batch_number, instruction_id, chat_completion, elapsed_time)

    torch.cuda.empty_cache()

    instruction_count += 1

Start fetching batch 1
Start fetching batch done...


<ipython-input-8-9ad68d3380dc>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '¿Bajo qué circunstancias los residentes en el extranjero deben pagar impuesto sobre la renta en México según el Artículo 1 de la Ley del Impuesto Sobre la Renta del 2024?' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[instruction_id, 'instruction'] = instruction


1 - LLM-ChatCompletion. InstructionId: 3878,  Elapsed-Time: 6.49 seconds
Palaven.SaveResponse. InstructionId: 3878,  Elapsed-Time: 1.13 seconds
2 - LLM-ChatCompletion. InstructionId: 3880,  Elapsed-Time: 2.25 seconds
Palaven.SaveResponse. InstructionId: 3880,  Elapsed-Time: 0.87 seconds
3 - LLM-ChatCompletion. InstructionId: 3882,  Elapsed-Time: 2.36 seconds
Palaven.SaveResponse. InstructionId: 3882,  Elapsed-Time: 0.83 seconds
4 - LLM-ChatCompletion. InstructionId: 3884,  Elapsed-Time: 3.82 seconds
Palaven.SaveResponse. InstructionId: 3884,  Elapsed-Time: 0.86 seconds
5 - LLM-ChatCompletion. InstructionId: 3886,  Elapsed-Time: 2.92 seconds
Palaven.SaveResponse. InstructionId: 3886,  Elapsed-Time: 0.83 seconds
6 - LLM-ChatCompletion. InstructionId: 3888,  Elapsed-Time: 1.30 seconds
Palaven.SaveResponse. InstructionId: 3888,  Elapsed-Time: 0.85 seconds
7 - LLM-ChatCompletion. InstructionId: 3894,  Elapsed-Time: 3.61 seconds
Palaven.SaveResponse. InstructionId: 3894,  Elapsed-Time: 0.83 

<ipython-input-8-9ad68d3380dc>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '¿Para calcular la modificación en el valor de bienes o operaciones en un periodo de un mes, qué factor de ajuste se utiliza de acuerdo al Artículo 6 de la Ley del Impuesto Sobre la Renta del 2024?' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[instruction_id, 'instruction'] = instruction


1 - LLM-ChatCompletion. InstructionId: 4017,  Elapsed-Time: 3.61 seconds
Palaven.SaveResponse. InstructionId: 4017,  Elapsed-Time: 0.86 seconds
2 - LLM-ChatCompletion. InstructionId: 4025,  Elapsed-Time: 4.15 seconds
Palaven.SaveResponse. InstructionId: 4025,  Elapsed-Time: 0.83 seconds
3 - LLM-ChatCompletion. InstructionId: 4027,  Elapsed-Time: 3.32 seconds
Palaven.SaveResponse. InstructionId: 4027,  Elapsed-Time: 0.83 seconds
4 - LLM-ChatCompletion. InstructionId: 4029,  Elapsed-Time: 3.75 seconds
Palaven.SaveResponse. InstructionId: 4029,  Elapsed-Time: 0.83 seconds
5 - LLM-ChatCompletion. InstructionId: 4031,  Elapsed-Time: 5.60 seconds
Palaven.SaveResponse. InstructionId: 4031,  Elapsed-Time: 0.83 seconds
6 - LLM-ChatCompletion. InstructionId: 4033,  Elapsed-Time: 3.96 seconds
Palaven.SaveResponse. InstructionId: 4033,  Elapsed-Time: 0.85 seconds
7 - LLM-ChatCompletion. InstructionId: 4035,  Elapsed-Time: 3.74 seconds
Palaven.SaveResponse. InstructionId: 4035,  Elapsed-Time: 0.85 

...

1 - LLM-ChatCompletion. InstructionId: 7433,  Elapsed-Time: 5.64 seconds
Palaven.SaveResponse. InstructionId: 7433,  Elapsed-Time: 1.02 seconds
2 - LLM-ChatCompletion. InstructionId: 7435,  Elapsed-Time: 4.86 seconds
Palaven.SaveResponse. InstructionId: 7435,  Elapsed-Time: 1.03 seconds
3 - LLM-ChatCompletion. InstructionId: 7437,  Elapsed-Time: 4.12 seconds
Palaven.SaveResponse. InstructionId: 7437,  Elapsed-Time: 1.04 seconds
4 - LLM-ChatCompletion. InstructionId: 7439,  Elapsed-Time: 4.77 seconds
Palaven.SaveResponse. InstructionId: 7439,  Elapsed-Time: 1.00 seconds
5 - LLM-ChatCompletion. InstructionId: 7441,  Elapsed-Time: 4.72 seconds
Palaven.SaveResponse. InstructionId: 7441,  Elapsed-Time: 1.03 seconds
6 - LLM-ChatCompletion. InstructionId: 7451,  Elapsed-Time: 12.98 seconds
Palaven.SaveResponse. InstructionId: 7451,  Elapsed-Time: 1.02 seconds
7 - LLM-ChatCompletion. InstructionId: 7453,  Elapsed-Time: 5.12 seconds
Palaven.SaveResponse. InstructionId: 7453,  Elapsed-Time: 0.83

<ipython-input-8-9ad68d3380dc>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Sumariza el artículo 169 de la Ley del Impuesto Sobre la Renta de 2024' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[instruction_id, 'instruction'] = instruction


1 - LLM-ChatCompletion. InstructionId: 7577,  Elapsed-Time: 6.55 seconds
Palaven.SaveResponse. InstructionId: 7577,  Elapsed-Time: 0.86 seconds
2 - LLM-ChatCompletion. InstructionId: 7579,  Elapsed-Time: 2.88 seconds
Palaven.SaveResponse. InstructionId: 7579,  Elapsed-Time: 0.81 seconds
3 - LLM-ChatCompletion. InstructionId: 7581,  Elapsed-Time: 4.69 seconds
Palaven.SaveResponse. InstructionId: 7581,  Elapsed-Time: 0.81 seconds
4 - LLM-ChatCompletion. InstructionId: 7583,  Elapsed-Time: 1.71 seconds
Palaven.SaveResponse. InstructionId: 7583,  Elapsed-Time: 0.83 seconds
5 - LLM-ChatCompletion. InstructionId: 7585,  Elapsed-Time: 2.05 seconds
Palaven.SaveResponse. InstructionId: 7585,  Elapsed-Time: 0.84 seconds
6 - LLM-ChatCompletion. InstructionId: 7587,  Elapsed-Time: 2.30 seconds
Palaven.SaveResponse. InstructionId: 7587,  Elapsed-Time: 0.83 seconds
7 - LLM-ChatCompletion. InstructionId: 7589,  Elapsed-Time: 2.20 seconds
Palaven.SaveResponse. InstructionId: 7589,  Elapsed-Time: 0.85 

<ipython-input-8-9ad68d3380dc>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '¿Los residentes en México y los residentes en el extranjero con establecimiento permanente en el país están obligados a pagar impuestos por los ingresos sujetos a regímenes fiscales preferentes obtenidos a través de entidades extranjeras en las que participen?' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[instruction_id, 'instruction'] = instruction


1 - LLM-ChatCompletion. InstructionId: 7712,  Elapsed-Time: 6.82 seconds
Palaven.SaveResponse. InstructionId: 7712,  Elapsed-Time: 1.03 seconds
2 - LLM-ChatCompletion. InstructionId: 7714,  Elapsed-Time: 5.81 seconds
Palaven.SaveResponse. InstructionId: 7714,  Elapsed-Time: 1.00 seconds
3 - LLM-ChatCompletion. InstructionId: 7716,  Elapsed-Time: 9.97 seconds
Palaven.SaveResponse. InstructionId: 7716,  Elapsed-Time: 1.03 seconds
4 - LLM-ChatCompletion. InstructionId: 7725,  Elapsed-Time: 13.97 seconds
Palaven.SaveResponse. InstructionId: 7725,  Elapsed-Time: 1.03 seconds
5 - LLM-ChatCompletion. InstructionId: 7727,  Elapsed-Time: 5.84 seconds
Palaven.SaveResponse. InstructionId: 7727,  Elapsed-Time: 1.03 seconds
6 - LLM-ChatCompletion. InstructionId: 7729,  Elapsed-Time: 4.99 seconds
Palaven.SaveResponse. InstructionId: 7729,  Elapsed-Time: 1.05 seconds
7 - LLM-ChatCompletion. InstructionId: 7731,  Elapsed-Time: 5.64 seconds
Palaven.SaveResponse. InstructionId: 7731,  Elapsed-Time: 1.00

### 4. Evaluation metrics

### 4.1. BERTScore

In [4]:
import pandas as pd

def build_responses_df():
  dataframe = pd.DataFrame(columns=['instruction_id',
    'evaluation_session_id', 'dataset_id', 'batch_size', 'large_language_model',
    'device_info', 'excercise_type', 'batch_number', 'instruction',
    'response', 'category', 'response_to_evaluate', 'elapsed_time'])

  dataframe['evaluation_session_id'] = dataframe['evaluation_session_id'].astype('object')
  dataframe['dataset_id'] = dataframe['dataset_id'].astype('object')
  dataframe['large_language_model'] = dataframe['large_language_model'].astype('object')
  dataframe['device_info'] = dataframe['device_info'].astype('object')
  dataframe['excercise_type'] = dataframe['excercise_type'].astype('object')
  dataframe['instruction'] = dataframe['instruction'].astype('object')
  dataframe['response'] = dataframe['response'].astype('object')
  dataframe['category'] = dataframe['category'].astype('object')
  dataframe['response_to_evaluate'] = dataframe['response_to_evaluate'].astype('object')

  dataframe.set_index('instruction_id', inplace=True)

  return dataframe


def add_response_to_df(dataframe, response):
  dataframe.loc[response['instructionId'], 'evaluation_session_id'] = response['evaluationSessionId']
  dataframe.loc[response['instructionId'], 'dataset_id'] = response['datasetId']
  dataframe.loc[response['instructionId'], 'batch_size'] = response['batchSize']
  dataframe.loc[response['instructionId'], 'large_language_model'] = response['largeLanguageModel']
  dataframe.loc[response['instructionId'], 'device_info'] = response['deviceInfo']
  dataframe.loc[response['instructionId'], 'excercise_type'] = response['evaluationExercise']
  dataframe.loc[response['instructionId'], 'batch_number'] = response['batchNumber']
  dataframe.loc[response['instructionId'], 'instruction'] = response['instruction']
  dataframe.loc[response['instructionId'], 'response'] = response['response']
  dataframe.loc[response['instructionId'], 'category'] = response['category']
  dataframe.loc[response['instructionId'], 'response_to_evaluate'] = response['llmResponseToEvaluate']
  dataframe.loc[response['instructionId'], 'elapsed_time'] = response['elapsedTime']

In [ ]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 971.4 kB/s eta 0:00:00


In [ ]:
from bert_score import score as bert_score

for batch_number in range(1, 29):

  model_responses_df = build_responses_df()

  print(f'Palaven. Bertscore evaluation. Start processing batch: {batch_number}')

  model_responses = palaven_api.fetch_model_responses(evaluation_session_id, 'llmfinetunedrag', batch_number)

  for item in model_responses:
    add_response_to_df(model_responses_df, item)

  references = model_responses_df['response'].tolist()
  candidates = model_responses_df['response_to_evaluate'].tolist()

  accuracy, recall, f1 = bert_score(candidates, references, lang='es', verbose=True, device='cuda')

  accuracy = accuracy.tolist()
  recall = recall.tolist()
  f1 = f1.tolist()

  average_accuracy = sum(accuracy) / len(accuracy)
  average_recall = sum(recall) / len(recall)
  average_f1 = sum(f1) / len(f1)

  print(f'Palaven. Bertscore evaluation. Batch: {batch_number}. Average accuracy: {average_accuracy}. Average recall: {average_recall}. Average F1: {average_f1}')

  palaven_api.save_bert_score_metrics(evaluation_session_id, 'llmfinetunedrag', batch_number, average_accuracy, average_recall, average_f1)

  print(f'Palaven. Bertscore evaluation. Posted batch {batch_number}')

Palaven. Bertscore evaluation. Start processing batch: 1
calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.86 seconds, 26.88 sentences/sec
Palaven. Bertscore evaluation. Batch: 1. Average accuracy: 0.7768898093700409. Average recall: 0.8239548087120057. Average F1: 0.7985292959213257
Palaven. Bertscore evaluation. Posted batch 1
Palaven. Bertscore evaluation. Start processing batch: 2
calculating scores...
computing bert embedding.


...

done in 0.32 seconds, 154.23 sentences/sec
Palaven. Bertscore evaluation. Batch: 27. Average accuracy: 0.7797505235671998. Average recall: 0.8492556488513947. Average F1: 0.8118519186973572
Palaven. Bertscore evaluation. Posted batch 27
Palaven. Bertscore evaluation. Start processing batch: 28
calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.14 seconds, 94.37 sentences/sec
Palaven. Bertscore evaluation. Batch: 28. Average accuracy: 0.8292911419501672. Average recall: 0.8591858698771551. Average F1: 0.8433318229822012
Palaven. Bertscore evaluation. Posted batch 28


### 4.2. ROUGE Score (1,2,L)

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=4de28694102b6e60c344b58fd7cad33157a10c00bc5d5dd7f690677c65177fcd
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from rouge_score import rouge_scorer

for batch_number in range(1, 29):

  model_responses_df = build_responses_df()

  print(f'Palaven. ROUGE evaluation. Start processing batch: {batch_number}')
  model_responses = palaven_api.fetch_model_responses(evaluation_session_id, 'llmfinetunedrag', batch_number)

  for item in model_responses:
    add_response_to_df(model_responses_df, item)

  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

  r1_precision, r1_recall, r1_f1 = [], [], []
  r2_precision, r2_recall, r2_f1 = [], [], []
  rL_precision, rL_recall, rL_f1 = [], [], []

  for _, row in model_responses_df.iterrows():
    reference = row['response']
    candidate = row['response_to_evaluate']

    scores = scorer.score(reference, candidate)

    #ROUGE-1
    r1_precision.append(scores['rouge1'].precision)
    r1_recall.append(scores['rouge1'].recall)
    r1_f1.append(scores['rouge1'].fmeasure)

    #ROUGE-2
    r2_precision.append(scores['rouge2'].precision)
    r2_recall.append(scores['rouge2'].recall)
    r2_f1.append(scores['rouge2'].fmeasure)

    #ROUGE-L
    rL_precision.append(scores['rougeL'].precision)
    rL_recall.append(scores['rougeL'].recall)
    rL_f1.append(scores['rougeL'].fmeasure)


  average_r1_precision = sum(r1_precision) / len(r1_precision)
  average_r1_recall = sum(r1_recall) / len(r1_recall)
  average_r1_f1 = sum(r1_f1) / len(r1_f1)

  average_r2_precision = sum(r2_precision) / len(r2_precision)
  average_r2_recall = sum(r2_recall) / len(r2_recall)
  average_r2_f1 = sum(r2_f1) / len(r2_f1)

  average_rL_precision = sum(rL_precision) / len(rL_precision)
  average_rL_recall = sum(rL_recall) / len(rL_recall)
  average_rL_f1 = sum(rL_f1) / len(rL_f1)

  rouge_metrics = []

  rouge_metrics.append({
      'rougeScoreType': 'rouge1',
      'batchNumber': batch_number,
      'precision': average_r1_precision,
      'recall': average_r1_recall,
      'f1': average_r1_f1
  });

  rouge_metrics.append({
      'rougeScoreType': 'rouge2',
      'batchNumber': batch_number,
      'precision': average_r2_precision,
      'recall': average_r2_recall,
      'f1': average_r2_f1
  });

  rouge_metrics.append({
      'rougeScoreType': 'rougeL',
      'batchNumber': batch_number,
      'precision': average_rL_precision,
      'recall': average_rL_recall,
      'f1': average_rL_f1
  });

  print(rouge_metrics)

  print(f'Palaven. ROUGE evaluation finished for batch: {batch_number}.')

  palaven_api.save_rouge_metrics(evaluation_session_id, 'llmfinetunedrag', batch_number, rouge_metrics)

  print(f'Palaven. Bertscore evaluation. Posted batch {batch_number}')

Palaven. ROUGE evaluation. Start processing batch: 1
[{'rougeScoreType': 'rouge1', 'batchNumber': 1, 'precision': 0.4684774919888217, 'recall': 0.6915906500512645, 'f1': 0.5181180072162592}, {'rougeScoreType': 'rouge2', 'batchNumber': 1, 'precision': 0.3421192096073171, 'recall': 0.48039003556497856, 'f1': 0.3800218640674516}, {'rougeScoreType': 'rougeL', 'batchNumber': 1, 'precision': 0.4174152728463804, 'recall': 0.6189775816800154, 'f1': 0.4618079036245441}]
Palaven. ROUGE evaluation finished for batch: 1.
Palaven. Bertscore evaluation. Posted batch 1
Palaven. ROUGE evaluation. Start processing batch: 2
[{'rougeScoreType': 'rouge1', 'batchNumber': 2, 'precision': 0.5474899788572336, 'recall': 0.761734852104298, 'f1': 0.5966189044350994}, {'rougeScoreType': 'rouge2', 'batchNumber': 2, 'precision': 0.44459671123971306, 'recall': 0.6249191274917981, 'f1': 0.49097433411934666}, {'rougeScoreType': 'rougeL', 'batchNumber': 2, 'precision': 0.4938824784527797, 'recall': 0.6857895974558985, 

### 4.3. BLEU

In [5]:
from nltk.translate.bleu_score import sentence_bleu

for batch_number in range(1, 29):

    model_responses_df = build_responses_df()

    print(f'Palaven. BLEU evaluation. Start processing batch: {batch_number}')

    model_responses = palaven_api.fetch_model_responses(evaluation_session_id, 'llmfinetunedrag', batch_number)

    for item in model_responses:
        add_response_to_df(model_responses_df, item)

    references = model_responses_df['response'].tolist()
    candidates = model_responses_df['response_to_evaluate'].tolist()

    bleu_scores = []

    for candidate, reference in zip(candidates, references):
        reference_tokens = [reference.split()]
        candidate_tokens = candidate.split()
        bleu_score = sentence_bleu(reference_tokens, candidate_tokens)
        bleu_scores.append(bleu_score)

    average_bleu = sum(bleu_scores) / len(bleu_scores)

    print(f'Palaven. BLEU evaluation. Batch: {batch_number}. Average BLEU score: {average_bleu}')

    palaven_api.save_bleu_score_metrics(evaluation_session_id, 'llmfinetunedrag', batch_number, average_bleu)

    print(f'Palaven. BLEU evaluation. Posted batch {batch_number}')

Palaven. BLEU evaluation. Start processing batch: 1


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Palaven. BLEU evaluation. Batch: 1. Average BLEU score: 0.24608320713895984
Palaven. BLEU evaluation. Posted batch 1
Palaven. BLEU evaluation. Start processing batch: 2


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Palaven. BLEU evaluation. Batch: 2. Average BLEU score: 0.35915124674546317
Palaven. BLEU evaluation. Posted batch 2
Palaven. BLEU evaluation. Start processing batch: 3


...

Palaven. BLEU evaluation. Batch: 25. Average BLEU score: 0.2919342492942466
Palaven. BLEU evaluation. Posted batch 25
Palaven. BLEU evaluation. Start processing batch: 26


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Palaven. BLEU evaluation. Batch: 26. Average BLEU score: 0.2400982006000456
Palaven. BLEU evaluation. Posted batch 26
Palaven. BLEU evaluation. Start processing batch: 27


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Palaven. BLEU evaluation. Batch: 27. Average BLEU score: 0.27571894761257554
Palaven. BLEU evaluation. Posted batch 27
Palaven. BLEU evaluation. Start processing batch: 28
Palaven. BLEU evaluation. Batch: 28. Average BLEU score: 0.38504342359649585
Palaven. BLEU evaluation. Posted batch 28
